In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import json

import tensorflow as tf

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
import tqdm

from scipy.stats import mode
import scipy.misc
import matplotlib.ticker as plticker

import seaborn as sns

from PIL import Image

import time

Using TensorFlow backend.


In [4]:
googlenet = models.InceptionV1()
googlenet.load_graphdef()

In [5]:
googlenet.layers

[Layer (belonging to InceptionV1) <conv2d0: 64> ([{'conv'}]),
 Layer (belonging to InceptionV1) <conv2d1: 64> ([{'conv'}]),
 Layer (belonging to InceptionV1) <conv2d2: 192> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed3a: 256> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed3b: 480> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4a: 508> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4b: 512> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4c: 512> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4d: 528> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4e: 832> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed5a: 832> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed5b: 1024> ([{'conv'}]),
 Layer (belonging to InceptionV1) <head0_bottleneck: 128> ([{'conv'}]),
 Layer (belonging to InceptionV1) <nn0: 1024> ([{'dense'}]),
 Layer (belonging to InceptionV1) <softmax0: 1008> ([{'dense'}]),
 Layer (belonging to InceptionV1) <head1_bottl

In [6]:
layers = {
'mixed3a': 256,
'mixed3b': 480,
'mixed4a': 508,
'mixed4b': 512,
'mixed4c': 512,
'mixed4d': 528,
'mixed4e': 832,
'mixed5a': 832,
'mixed5b': 1024
}

In [12]:
layer = 'mixed3b'
num_of_classes = 1000
num_of_chanels_in_layer = layers[layer]
prob_mass_threshold = 0.01
batch = 200

In [ ]:
def _parse_function(example_proto, image_size=224):
    def _bytes_feature(value):
        return tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[value]))

    def _int64_feature(value):
        return tf.train.Feature(
            int64_list=tf.train.Int64List(value=[value]))
    
    feature_set = {
        'image/filename': tf.FixedLenFeature([], tf.string),
        'image/encoded': tf.FixedLenFeature([], tf.string),
        'image/height': tf.FixedLenFeature([], tf.int64),
        'image/width': tf.FixedLenFeature([], tf.int64),
        'image/channels': tf.FixedLenFeature([], tf.int64),
        'image/class/label': tf.FixedLenFeature([], tf.int64),
        'image/class/synset': tf.FixedLenFeature([], tf.string)}
  
    parsed_features = tf.parse_single_example(example_proto, feature_set)
    
    image_id = parsed_features['image/filename']
    
    label = parsed_features['image/class/label']
    
    width = parsed_features['image/width']
    height = parsed_features['image/height']
    channels = parsed_features['image/channels']
    
    synset = parsed_features['image/class/synset']
    
    image = parsed_features['image/encoded']
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, tf.constant([image_size, image_size]))
    
    return image, label, synset


filenames = glob.glob('/media/fred/lime/imagenet-tf-records/*') 
# filenames = glob.glob('test-images/imagenet-tf-records/*') 

print(len(filenames))

M = np.zeros([num_of_classes, num_of_chanels_in_layer], dtype=int)

with tf.Graph().as_default():
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(_parse_function)
    dataset = dataset.map(lambda img, lab, syn: (preprocess_input(img), lab, syn))
    dataset = dataset.batch(batch)

    iterator = dataset.make_one_shot_iterator()
    t_preprocessed_images, t_labels, t_synsets = iterator.get_next()

    T = render.import_model(googlenet, t_preprocessed_images, None)
    t_imgs_acts = T(layer)

    progress_counter = 0
    with tf.Session() as sess:
        start = time.time()
        
        try:
            with tqdm.tqdm(total=1281167, unit='imgs') as pbar:
                while(True):
                    progress_counter += 1
                    imgs_acts, labels, synsets = sess.run([t_imgs_acts,t_labels,t_synsets])
                    
                    # no sess.run after this
                    # python code here on out
                    for i in range(imgs_acts.shape[0]):
                        acts_max = np.zeros((imgs_acts.shape[-1]))
                        
                        for j in range(imgs_acts[i].shape[-1]):
                            acts_max[j] = np.max(imgs_acts[i,:,:,j])
                                                
                        # METHOD 2: prob mass
                        top_channels = []
                        working_acts_max = acts_max/np.sum(acts_max)
                        prob_mass = 0
                        sorted_working_acts_max, sorted_inds = (list(t) for t in zip(*sorted(zip(working_acts_max, list(range(working_acts_max.shape[0]))), reverse=True)))
                        k = 0
                        while prob_mass < prob_mass_threshold:
                            top_channels.append(sorted_inds[k])
                            prob_mass += sorted_working_acts_max[k]
                            k += 1
                        for top_channel in top_channels:
                            M[labels[i]-1][top_channel] += 1
#                         print(synsets[i], labels[i])
    
                    pbar.update(len(labels))

        except tf.errors.OutOfRangeError:
            pass
        
        end = time.time()
        print(end - start)
        print(progress_counter)
        print(progress_counter*batch)

  0%|          | 0/1281167 [00:00<?, ?imgs/s]

1024


 75%|███████▌  | 964400/1281167 [1:28:40<29:25, 179.37imgs/s]

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(M)
plt.colorbar()
plt.show()

In [ ]:
np.savetxt('layer-matrices/' + 'M-march12test-' + layer + '.csv', M, delimiter=',', fmt='%i')